# Download the latest data
There are up to four calls that need to be made to download or update the pedestrian safety data.
- Mapillary's road data
- Mapillary's signage data
- LAPD's collision data
- The neighborhood council boundaries shapefile

In [1]:
import configparser
import pandas as pd
import datetime
import os
import sys

# Load the mapillary client module
try:
    os.chdir(os.path.dirname(os.path.realpath(__file__)))
except:
    pass
mapillarywrapperdir = '../../311-data/mapillarywrapper/'
sys.path.append(mapillarywrapperdir)
from mapillarywrapper.client import client


config = configparser.ConfigParser()
config.read(mapillarywrapperdir + 'mapillary.cfg')
CLIENT_ID = config['CLIENT-ID']['CLIENT_ID']
mapillaryclient = client.MapClient(CLIENT_ID)



Set the save locations of the raw data csv files.

In [2]:
rawdatadir = '../Data/1-raw-data/'
rawroaddatacsv = rawdatadir + 'raw-road-data.csv'
rawsigndatacsv = rawdatadir + 'raw-sign-data.csv'
rawcollisiondatacsv = rawdatadir + 'raw-collision-data.csv'
neighborhoodcouncilshapes = rawdatadir + 'neighborhoodcouncils.geojson'

## Bounding box

This is the latitude/longitude box that encompasses the City of LA according to https://boundingbox.klokantech.com

In [3]:
laLowerLeft = [33.703622, -118.668187]
laUpperRight = [34.337306, -118.155295]

## Merging method
This will combine dataframes of the same dataset downloaded at different times or with different filters.

In [4]:
def combinedataframes(
    leftdf: pd.DataFrame, rightdf: pd.DataFrame, keycolumn: str
):
  """Combine dataframes of the same dataset downloaded at different times or with different filters."""

  # Use the keycolumn as the index with a different name, to differentiate them.
  leftreindexed = leftdf.set_index(keycolumn, drop=False)
  leftreindexed.index.name = 'indexkeys'
  rightreindexed = rightdf.set_index(keycolumn, drop=False)
  rightreindexed.index.name = 'indexkeys'

  # Merge the keys, creating empty key-only rows for all new data
  allrows = leftreindexed.merge(
      rightreindexed[keycolumn], left_index=True, right_index=True, how='outer'
  ).drop(columns=[keycolumn + '_x', keycolumn + '_y'])

  # Update all rows with the newest data
  allrows.update(rightreindexed)
  
  # copy the keys from the index back into a regular column
  allrows[keycolumn] = allrows.index
  allrows.reset_index(drop=True, inplace=True)
  return allrows




# Download data from mapillary
If mapillary data is saved locally, download the latest. If it isn't, download everything.

In [5]:
def frommapillarydateformat(isostring: str):
  return datetime.datetime.fromisoformat(isostring.strip('Z'))


def tomapillarydateformat(dt: datetime.datetime):
  return dt.isoformat('T', 'milliseconds') + 'Z'

for [mapillarylayer, csvlocation] in [
  ['points', rawroaddatacsv], ['trafficsigns', rawsigndatacsv]
]:
  savefileexists = os.path.isfile(csvlocation)
  if savefileexists:
    previousdownload = pd.read_csv(csvlocation)
    previousdownload['last_seen_at']
    mostrecentdata = previousdownload['last_seen_at'].map(
      frommapillarydateformat
    ).max()
    datacutoffdate = mostrecentdata - datetime.timedelta(days=7)

    newmapillarydata = pd.DataFrame(
      mapillaryclient.trafficinfo(
        laLowerLeft,
        laUpperRight,
        perpage=1000,
        layer=mapillarylayer,
        startlastseenat=tomapillarydateformat(datacutoffdate.to_pydatetime())
      )
    )

    fulldataset = combinedataframes(previousdownload, newmapillarydata, 'key')

  else:
    fulldataset = pd.DataFrame(
      mapillaryclient.trafficinfo(
        laLowerLeft,
        laUpperRight,
        perpage=1000,
        layer=mapillarylayer,
      )
    )

  fulldataset.to_csv(csvlocation, index=False)
  print(f'saved to {csvlocation}')

Requesting data from Mapillary API
https://a.mapillary.com/v3/map_features?layers=points&bbox=-118.668187,33.703622,-118.155295,34.337306&start_last_seen_at=2021-06-01T23:47:32.611Z&per_page=1000&client_id=c1hsc1BzRGY2ZXBQR1FCY1JIaUJRQTpjOTZmZThjNjkwYmI5MTQx&sort_by=key
Pages found (with up to 1000 results per page):
1
Pages of results loaded:
1
saved to ../Data/1-raw-data/raw-road-data.csv
Requesting data from Mapillary API
https://a.mapillary.com/v3/map_features?layers=trafficsigns&bbox=-118.668187,33.703622,-118.155295,34.337306&start_last_seen_at=2021-06-01T23:45:25.561Z&per_page=1000&client_id=c1hsc1BzRGY2ZXBQR1FCY1JIaUJRQTpjOTZmZThjNjkwYmI5MTQx&sort_by=key
Pages found (with up to 1000 results per page):
1
Pages of results loaded:
1
saved to ../Data/1-raw-data/raw-sign-data.csv


# Download LAPD Traffic collision data
tk